In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 21.6 MB/s eta 0:00:00


In [ ]:
pip install pymongo google-generativeai python-dotenv tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.1 MB/s eta 0:00:00


In [ ]:
from pymongo import MongoClient
import google.generativeai as genai
from tqdm import tqdm
import re
import json
import ast

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env file

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
MONGO_URI = os.getenv('MONGO_URI')

# Optional: Add validation
if not MONGO_URI or not GOOGLE_API_KEY:
    raise ValueError("Missing required environment variables")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# MongoDB Configuration

DB_NAME = "bible_rag_db"
client = MongoClient(MONGO_URI)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Unlocking the Bible ( PDFDrive )_105037.pdf to Unlocking the Bible ( PDFDrive )_105037.pdf


### Extracting Structured Text from a PDF and Saving as JSON

This section processes a PDF file by extracting readable content within a specified page range, skipping structural or title-only pages, and organizing the data with metadata such as section, subsection, and chapter title. The output is a clean, structured JSON file suitable for downstream text analysis or application integration.

In [ ]:
import fitz  # PyMuPDF
import json
import re

def create_json_from_pdf(pdf_path, json_path, start_page, end_page):
    """
    Reads a PDF, processes its content within a specific page range,
    and exports the data as a JSON file, excluding section/subsection title pages
    and pages with less than three lines of content.

    Args:
        pdf_path (str): The file path to the input PDF.
        json_path (str): The file path for the output JSON file.
        start_page (int): The page number to start processing from.
        end_page (int): The page number to stop processing at.
    """
    # These variables will keep track of the current context as we loop through pages
    current_testament = "Unknown"
    current_subsection = "Unknown"
    current_chapter_title = "Unknown"

    # This list will hold the data for each page
    all_page_data = []

    print(f"Opening and processing '{pdf_path}'...")
    print(f"Targeting pages from {start_page} to {end_page}.")

    # Use a 'with' statement for safe file handling
    with fitz.open(pdf_path) as doc:
        # Iterate through each page of the PDF
        for page_num, page in enumerate(doc, 1):

            # --- ADJUSTMENT 1: Process only the specified page range ---
            if not (start_page <= page_num <= end_page):
                continue  # Skip any page outside the desired range

            # Extract all text from the current page
            text = page.get_text("text").strip()

            # If a page is completely empty, skip it
            if not text:
                continue

            # --- NEW ADJUSTMENT: Skip if content has less than three lines ---
            # Split the text by newline characters and check the number of lines.
            # If it's less than 3, it's considered too short for meaningful content.


            # Count the number of sentences based on '.', '?', or '!' followed by a space or end of line
            # Remove all whitespace characters (spaces, tabs, newlines)


            # --- ADJUSTMENT 2: Detect structural pages and skip saving them ---

            # 1. Check for Major Section (Testament) pages
            if re.fullmatch(r"I\s*\nOLD\s+TESTAMENT", text, re.IGNORECASE):
                current_testament = "The Old Testament"
                # This is a structural page, so we update the context and skip to the next page
                continue
            elif re.fullmatch(r"II\s*\nNEW\s+TESTAMENT", text, re.IGNORECASE):
                current_testament = "The New Testament"
                # This is also a structural page, skip saving it
                continue

            # 2. Check for Subsection pages
            lines = text.split('\n')
            # Heuristic: first line is all caps, not a chapter title, and other lines look like a list.
            if len(lines) > 2 and lines[0].isupper() and re.search(r"^\d+\.", lines[2].strip()):
                current_subsection = lines[0].strip()
                current_chapter_title = "Introduction to Subsection"
                # This is a subsection contents page, skip saving it
                continue

            # --- If the script reaches this point, it's a content page that should be saved ---

            # 3. Check for the start of a specific chapter's content on the current page
            # Example: "7. OVERVIEW OF THE OLD\n\nTESTAMENT"
            chapter_match = re.match(r"(?s)^\s*(\d+\.?\s+[A-Z0-9\s,&]+?)\n", text)
            if chapter_match:
                # Extract and clean up the matched title
                current_chapter_title = re.sub(r'\s+', ' ', chapter_match.group(1)).strip()
            text_no_spaces = ''.join(text.split())

            if len(text_no_spaces) < 360:  # 4 average-length sentences ≈ 360 characters (without spaces)
                print(f"Skipping page {page_num} due to insufficient content (less than 4 sentences' worth of text).")
                continue  # Skip to the next page

            # --- Create the JSON object for the current (content) page ---
            page_chunk = {
                "page": page_num,
                "section": current_testament,
                "subsection": current_subsection,
                "chapter_title": current_chapter_title,
                "text": text
            }

            all_page_data.append(page_chunk)

    # --- Write the collected data to a JSON file ---
    print(f"Writing data for {len(all_page_data)} pages to '{json_path}'...")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_page_data, f, indent=4, ensure_ascii=False)

    print("Processing complete.")


# --- Main execution ---
if __name__ == "__main__":
    # Define the name of your PDF and the desired output JSON file
    pdf_file = "unlockingthebible.pdf"
    json_file = "unlockingthebible_chunks.json"

    # Define the page range you want to process
    START_PAGE_TO_PROCESS = 21
    END_PAGE_TO_PROCESS = 1318

    create_json_from_pdf(pdf_file, json_file, START_PAGE_TO_PROCESS, END_PAGE_TO_PROCESS)

Opening and processing 'unlockingthebible.pdf'...
Targeting pages from 21 to 1318.
Skipping page 35 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 142 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 164 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 214 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 276 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 318 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 419 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 420 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 453 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 465 due to insufficient content (less than 4 sentences' worth of text).
Skipping page 479 due to insufficient content (less than 4

### Embedding Text Chunks and Storing in MongoDB for Semantic Search

This section loads structured JSON data, generates embeddings using Google's text embedding model, and stores the enriched documents in a MongoDB collection. It includes steps for connecting to the database, handling data integrity, and preparing the data for use in retrieval-augmented generation (RAG) or semantic search systems.

In [ ]:

import os
import json
import pymongo
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm
from google.colab import files


DB_NAME = "bible_rag_db"
COLLECTION_NAME = "commentary_chunks"

# Google AI Configuration
EMBEDDING_MODEL = "models/text-embedding-004" # The model for generating embeddings

# Input JSON file
INPUT_JSON_FILE = "unlockingthebible_chunks.json"

# --- Main Script ---

def connect_to_mongodb():
    """Establishes a connection to MongoDB and returns the collection object."""
    try:
        print("Connecting to MongoDB...")
        client = pymongo.MongoClient(MONGO_URI)
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print("MongoDB connection successful.")
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        return client, collection
    except pymongo.errors.ConnectionFailure as e:
        print(f"Could not connect to MongoDB: {e}")
        raise

def load_json_data(file_path):
    """Loads and returns data from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            print(f"Loading data from {file_path}...")
            data = json.load(f)
            print(f"Successfully loaded {len(data)} chunks.")
            return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid JSON file.")
        raise

def generate_embedding(text):
    """Generates an embedding for the given text using Google AI."""
    try:
        # The 'task_type' is crucial for retrieval/semantic search tasks
        result = genai.embed_content(
            model=EMBEDDING_MODEL,
            content=text,
            task_type="RETRIEVAL_DOCUMENT"
        )
        return result['embedding']
    except Exception as e:
        print(f"An error occurred during embedding generation: {e}")
        return None

def main():
    """Main function to orchestrate the embedding and insertion process."""
    # Configure the Google AI client
    genai.configure(api_key=GOOGLE_API_KEY)

    mongo_client = None
    try:
        # 1. Connect to MongoDB
        mongo_client, collection = connect_to_mongodb()

        # Optional: Clear the collection to avoid duplicates on re-runs
        # If you want to append data, comment out the next two lines.
        print(f"Clearing existing documents in '{COLLECTION_NAME}' collection...")
        collection.delete_many({})

        # 2. Load the source data
        chunks_data = load_json_data(INPUT_JSON_FILE)

        # 3. Process each chunk and insert into MongoDB
        print("Starting to process and embed chunks...")
        documents_to_insert = []
        for chunk in tqdm(chunks_data, desc="Processing Chunks"):
            # Ensure the text is a non-empty string
            text_to_embed = chunk.get("text", "")
            if chunk["page"] < 21:
                continue
            if not isinstance(text_to_embed, str) or not text_to_embed.strip():
                print(f"Skipping chunk on page {chunk.get('page')} due to empty text.")
                continue

            # 4. Generate the embedding
            embedding = generate_embedding(text_to_embed)

            if embedding:
                # 5. Prepare the document for MongoDB insertion
                # Note: 'subsection' is mapped to 'sub_section' as requested.
                document = {
                    "section": chunk.get("section"),
                    "sub_section": chunk.get("subsection"), # Renaming the field
                    "chapter_title": chunk.get("chapter_title"),
                    "text": text_to_embed,
                    "embedding": embedding
                }
                documents_to_insert.append(document)

        # 6. Insert all prepared documents in a single batch operation
        if documents_to_insert:
            print(f"\nInserting {len(documents_to_insert)} documents into MongoDB...")
            collection.insert_many(documents_to_insert)
            print("All documents have been successfully inserted.")
        else:
            print("No documents were generated to insert.")

    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        # 7. Ensure the MongoDB connection is closed
        if mongo_client:
            mongo_client.close()
            print("MongoDB connection closed.")

if __name__ == "__main__":
    main()

Connecting to MongoDB...
MongoDB connection successful.
Clearing existing documents in 'commentary_chunks' collection...
Loading data from unlockingthebible_chunks.json...
Successfully loaded 1254 chunks.
Starting to process and embed chunks...


Processing Chunks: 100%|██████████| 1254/1254 [38:25<00:00,  1.84s/it]



Inserting 1254 documents into MongoDB...
All documents have been successfully inserted.
MongoDB connection closed.


### Normalizing Chapter Titles in MongoDB with Custom Overrides and Regex

This section updates inconsistent or formatted chapter titles in the MongoDB collection. It applies predefined custom title overrides and uses regex to extract and standardize titles, ensuring uniformity across the dataset for improved readability and consistency in downstream applications.

In [ ]:

from pymongo import MongoClient
import re

# Custom title overrides
custom_titles = {
    "God Is Powerful": "Genesis",
    "Idolatry": "Deuteronomy",
    "Old Testament Use": "Matthew",
    "Paul": "Acts",
    "His Age": "1, 2, And 3 John",
    "The Possibilities": "1, 2, And 3 John"
}

mongo_client, collection = connect_to_mongodb()
# Fetch all documents with a chapter_title
documents = collection.find({"chapter_title": {"$exists": True}})

for doc in documents:
    original_title = doc["chapter_title"]

    # Check for custom overrides first
    if original_title in custom_titles:
        new_title = custom_titles[original_title]
    else:
        # Extract the title after number and dot (e.g., "58. REVELATION" -> "Revelation")
        match = re.match(r"^\d+\.\s+(.*)", original_title)
        if match:
            # Capitalize properly (title case)
            new_title = match.group(1).title()
        else:
            continue  # Skip if it doesn't match the expected format

    # Update in MongoDB if changed
    if new_title != original_title:
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"chapter_title": new_title}}
        )
        print(f"Updated: {original_title} → {new_title}")

Connecting to MongoDB...
MongoDB connection successful.
Updated: God Is Powerful → Genesis
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Idolatry → Deuteronomy
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew
Updated: Old Testament Use → Matthew


In [ ]:

import google.generativeai as genai

def print_gemini_model_names():
    """Prints the names of the GenerativeModel models available."""
    try:
        # List all available models
        for m in genai.list_models():
            # Check if the model supports content generation (is a GenerativeModel)
            if "generateContent" in m.supported_generation_methods:
                print(m.name)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # You might need to configure your API key here if not already set
    # genai.configure(api_key="YOUR_API_KEY")
    print_gemini_model_names()

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thi

### Extracting and Storing Bible Verse References from Theological Concepts

This section captures user-input theological summaries, uses a Gemini prompt to extract exact Bible verse references and their distribution by book, parses the structured JSON response, and stores the enriched data into MongoDB for future querying or analysis.

In [ ]:
import re

# ========= INPUT =========

concept = input("Enter theological concept title: ").strip()
print("Enter the theological summary (end with ENTER twice):")
lines = []
while True:
    line = input()
    if line == "":
        break
    lines.append(line)
summary_text = "\n".join(lines)

# ========= GEMINI PROMPT =========

prompt = f"""
From the following text, extract:

1. A list of **exact Bible verses** mentioned (e.g., "Genesis 1:1", "2 Timothy 3:16").
2. Then compute the **percentage of total verses** by book (e.g., "Genesis": 40.0).

Return a JSON object like:
{{
  "verses_mentioned": ["Genesis 1:1", "Exodus 31:17", "2 Timothy 3:16"],
  "book_reference_stats": {{
    "Genesis": 33.3,
    "Exodus": 33.3,
    "2 Timothy": 33.3
  }}
}}

Text:
{summary_text}
"""

response = model.generate_content(prompt)

# ========= PARSE GEMINI JSON SAFELY =========

raw_text = response.text.strip()

# Extract between triple backticks if present
match = re.search(r"```(?:json)?(.*?)```", raw_text, re.DOTALL)
if match:
    cleaned = match.group(1).strip()
else:
    cleaned = raw_text

# Fix single quotes to double quotes (assuming clean response)
cleaned = cleaned.replace("'", '"')

try:
    result_json = json.loads(cleaned)
    verses_mentioned = result_json.get("verses_mentioned", [])
    book_reference_stats = result_json.get("book_reference_stats", {})
except Exception as e:
    print("❌ Error parsing Gemini response:", e)
    verses_mentioned = []
    book_reference_stats = {}

# ========= STORE IN MONGODB =========

doc = {
    "concept": concept,
    "summary": summary_text,
    "verses_mentioned": verses_mentioned,
    "book_reference_stats": book_reference_stats
}

insert_result = collection.insert_one(doc)
print(f"✔️ Inserted document with _id: {insert_result.inserted_id}")Bible

Enter theological concept title: Eternity
Enter the theological summary (end with ENTER twice):
We teach that after the closing of the Millennium, the temporary release of  Satan, and the judgment of unbelievers (2 Thess. 1:9; Rev. 20:7–15), the saved  will enter the eternal state of glory with God, after which the elements of this  earth are to be dissolved (2 Pet. 3:10) and replaced with a new earth wherein  only righteousness dwells (Eph. 5:5; Rev. 20:15; 21–22). Following this, the  heavenly city will come down out of heaven (Rev. 21:2) and will be the dwelling  place of the saints, where they will enjoy forever fellowship with God and one  another (John 17:3; Rev. 21–22). Our Lord Jesus Christ, having fulfilled his  redemptive mission, will then deliver up the kingdom to God the Father (1 Cor.  15:23–28) that in all spheres the triune God may reign forever and ever (1 Cor.  15:28).

✔️ Inserted document with _id: 684c1f2bdc45f4c1ffcbf3bb


### Initializing the Bible Books Collection in MongoDB

This code sets up a `books` collection by defining metadata for all 66 books of the Bible. Each entry includes the book's name, testament (Old/New), category (e.g., Law, History, Gospel), and a unique ID. The collection is cleared before insertion to ensure a clean slate for consistent querying and data integrity.

In [ ]:


collection = db["books"]

# Define all 66 books with their metadata
books = [
    # Law (Pentateuch)
    {"id": 1, "name": "Genesis", "testament": "Old", "category": "Law"},
    {"id": 2, "name": "Exodus", "testament": "Old", "category": "Law"},
    {"id": 3, "name": "Leviticus", "testament": "Old", "category": "Law"},
    {"id": 4, "name": "Numbers", "testament": "Old", "category": "Law"},
    {"id": 5, "name": "Deuteronomy", "testament": "Old", "category": "Law"},

    # History
    {"id": 6, "name": "Joshua", "testament": "Old", "category": "History"},
    {"id": 7, "name": "Judges", "testament": "Old", "category": "History"},
    {"id": 8, "name": "Ruth", "testament": "Old", "category": "History"},
    {"id": 9, "name": "1 Samuel", "testament": "Old", "category": "History"},
    {"id": 10, "name": "2 Samuel", "testament": "Old", "category": "History"},
    {"id": 11, "name": "1 Kings", "testament": "Old", "category": "History"},
    {"id": 12, "name": "2 Kings", "testament": "Old", "category": "History"},
    {"id": 13, "name": "1 Chronicles", "testament": "Old", "category": "History"},
    {"id": 14, "name": "2 Chronicles", "testament": "Old", "category": "History"},
    {"id": 15, "name": "Ezra", "testament": "Old", "category": "History"},
    {"id": 16, "name": "Nehemiah", "testament": "Old", "category": "History"},
    {"id": 17, "name": "Esther", "testament": "Old", "category": "History"},

    # Poetry/Wisdom
    {"id": 18, "name": "Job", "testament": "Old", "category": "Poetry"},
    {"id": 19, "name": "Psalms", "testament": "Old", "category": "Poetry"},
    {"id": 20, "name": "Proverbs", "testament": "Old", "category": "Poetry"},
    {"id": 21, "name": "Ecclesiastes", "testament": "Old", "category": "Poetry"},
    {"id": 22, "name": "Song of Solomon", "testament": "Old", "category": "Poetry"},

    # Major Prophets
    {"id": 23, "name": "Isaiah", "testament": "Old", "category": "Major Prophet"},
    {"id": 24, "name": "Jeremiah", "testament": "Old", "category": "Major Prophet"},
    {"id": 25, "name": "Lamentations", "testament": "Old", "category": "Major Prophet"},
    {"id": 26, "name": "Ezekiel", "testament": "Old", "category": "Major Prophet"},
    {"id": 27, "name": "Daniel", "testament": "Old", "category": "Major Prophet"},

    # Minor Prophets
    {"id": 28, "name": "Hosea", "testament": "Old", "category": "Minor Prophet"},
    {"id": 29, "name": "Joel", "testament": "Old", "category": "Minor Prophet"},
    {"id": 30, "name": "Amos", "testament": "Old", "category": "Minor Prophet"},
    {"id": 31, "name": "Obadiah", "testament": "Old", "category": "Minor Prophet"},
    {"id": 32, "name": "Jonah", "testament": "Old", "category": "Minor Prophet"},
    {"id": 33, "name": "Micah", "testament": "Old", "category": "Minor Prophet"},
    {"id": 34, "name": "Nahum", "testament": "Old", "category": "Minor Prophet"},
    {"id": 35, "name": "Habakkuk", "testament": "Old", "category": "Minor Prophet"},
    {"id": 36, "name": "Zephaniah", "testament": "Old", "category": "Minor Prophet"},
    {"id": 37, "name": "Haggai", "testament": "Old", "category": "Minor Prophet"},
    {"id": 38, "name": "Zechariah", "testament": "Old", "category": "Minor Prophet"},
    {"id": 39, "name": "Malachi", "testament": "Old", "category": "Minor Prophet"},

    # Gospels
    {"id": 40, "name": "Matthew", "testament": "New", "category": "Gospel"},
    {"id": 41, "name": "Mark", "testament": "New", "category": "Gospel"},
    {"id": 42, "name": "Luke", "testament": "New", "category": "Gospel"},
    {"id": 43, "name": "John", "testament": "New", "category": "Gospel"},

    # History (Acts)
    {"id": 44, "name": "Acts", "testament": "New", "category": "History"},

    # Pauline Epistles
    {"id": 45, "name": "Romans", "testament": "New", "category": "Pauline Epistle"},
    {"id": 46, "name": "1 Corinthians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 47, "name": "2 Corinthians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 48, "name": "Galatians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 49, "name": "Ephesians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 50, "name": "Philippians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 51, "name": "Colossians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 52, "name": "1 Thessalonians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 53, "name": "2 Thessalonians", "testament": "New", "category": "Pauline Epistle"},
    {"id": 54, "name": "1 Timothy", "testament": "New", "category": "Pauline Epistle"},
    {"id": 55, "name": "2 Timothy", "testament": "New", "category": "Pauline Epistle"},
    {"id": 56, "name": "Titus", "testament": "New", "category": "Pauline Epistle"},
    {"id": 57, "name": "Philemon", "testament": "New", "category": "Pauline Epistle"},

    # General Epistles
    {"id": 58, "name": "Hebrews", "testament": "New", "category": "General Epistle"},
    {"id": 59, "name": "James", "testament": "New", "category": "General Epistle"},
    {"id": 60, "name": "1 Peter", "testament": "New", "category": "General Epistle"},
    {"id": 61, "name": "2 Peter", "testament": "New", "category": "General Epistle"},
    {"id": 62, "name": "1 John", "testament": "New", "category": "General Epistle"},
    {"id": 63, "name": "2 John", "testament": "New", "category": "General Epistle"},
    {"id": 64, "name": "3 John", "testament": "New", "category": "General Epistle"},
    {"id": 65, "name": "Jude", "testament": "New", "category": "General Epistle"},

    # Prophecy
    {"id": 66, "name": "Revelation", "testament": "New", "category": "Prophecy"},
]

# Clear collection before inserting (optional)
collection.delete_many({})

# Insert all books
collection.insert_many(books)

print("Inserted 66 books into the 'books' collection.")

Inserted 66 books into the 'books' collection.


### Generating Theological Summaries for Bible Books Using Gemini AI

This section retrieves each Bible book from MongoDB, compiles related commentary chunks, and sends a prompt to the Gemini 2.0 model to generate a structured JSON summary containing an overview, key scripture references, and theological context. The JSON response is cleaned, parsed, and then used to update the corresponding book records in MongoDB, enabling enhanced theological insights for future use.

In [ ]:


books_collection = db["books"]
commentary_collection = db["commentary_chunks"]
PROMPT_TEMPLATE = """
You are a biblical scholar and summarizer. Based on the following commentary for the book of {book_name}, do the following:

1. Write a 3–5 sentence **overview** of the book.
2. Identify and list 2 to 3 **key scripture references** with brief descriptions.
3. Write a 3–5 sentence **theological context** explaining the major doctrines and themes.

Commentary:
\"\"\"
{commentary_text}
\"\"\"

Return your response in a strict JSON format. The JSON object must contain the keys "overview", "key_scriptures", and "theological_context".
Do not include any text, explanations, or markdown formatting outside of the JSON object itself.
{{
  "overview": "...",
  "key_scriptures": [
    "...",
    "..."
  ],
  "theological_context": "..."
}}
"""

# === Revised Helper Function to Fix and Parse Gemini Output ===
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ` ```json ... ``` or ` ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace("“", '"').replace("”", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("‘", "'").replace("’", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# === Main Process (User's original logic) ===
for book in tqdm(books_collection.find()):
    book_name = book["name"]
    book_name_lower = book_name.lower()

    # Get all commentary chunks for this book (case-insensitive)
    chunks = commentary_collection.find({
        "$expr": { "$eq": [{ "$toLower": "$chapter_title" }, book_name_lower] }
    })

    all_text = " ".join(chunk["text"] for chunk in chunks)

    if not all_text.strip():
        print(f"⚠️ Skipping {book_name} - No commentary found.")
        continue

    # Format the prompt
    prompt = PROMPT_TEMPLATE.format(
        book_name=book_name,
        commentary_text=all_text[:200000] # Make sure your model's context window supports this
    )

    try:
        response = model.generate_content(prompt)
        raw_text = response.text.strip()

        print(f"\n📜 Raw Gemini response for {book_name}:\n{raw_text}\n")

        generated_fields = parse_gemini_json(raw_text)

        # === Update MongoDB ===
        books_collection.update_one(
            {"id": book["id"]},
            {"$set": {
                "overview": generated_fields["overview"],
                "key_scriptures": generated_fields["key_scriptures"],
                "theological_context": generated_fields["theological_context"]
            }}
        )

        print(f"✅ Updated: {book_name}")

    except Exception as e:
        print(f"❌ An error occurred processing {book_name}: {e}")
        continue

0it [00:00, ?it/s]


📜 Raw Gemini response for Genesis:
```json
{
  "overview": "Genesis, meaning \"beginnings,\" is the foundational book of the Bible, narrating the origins of the universe, humanity, sin, and God's initial covenant relationships. It sets the stage for the entire biblical narrative by explaining why the world is the way it is and how God intends to redeem it. The book covers a vast period from creation to the time of Joseph, focusing particularly on God's interactions with key figures like Adam, Noah, Abraham, Isaac, Jacob, and Joseph and how these figures are vital to God's plan for humanity.",
  "key_scriptures": [
    "Genesis 1:1 - \"In the beginning God created the heavens and the earth\" establishes God as the eternal creator, rejecting atheistic and polytheistic views.",
    "Genesis 3:15 - This verse contains the Protoevangelium, the first gospel, promising a future offspring of the woman (Jesus) who will ultimately defeat the serpent (Satan).",
    "Genesis 12:1-3 - God's call t

1it [00:22, 22.83s/it]

✅ Updated: Genesis

📜 Raw Gemini response for Exodus:
```json
{
  "overview": "Exodus recounts the Israelites' miraculous liberation from Egyptian slavery under the leadership of Moses. God demonstrates his power through a series of plagues, culminating in the Passover and the parting of the Red Sea. Following their escape, God establishes a covenant with Israel at Mount Sinai, giving them the Law, including the Ten Commandments, to govern their lives. The book details the construction of the Tabernacle, symbolizing God's presence among his people, and highlights the importance of gratitude and obedience in their relationship with him.",
  "key_scriptures": [
    {
      "reference": "Exodus 3:14",
      "description": "God reveals his name as 'Yahweh,' signifying his eternal and self-existent nature ('I AM WHO I AM'). This establishes a personal and intimate relationship between God and his people, emphasizing his unwavering presence and provision."
    },
    {
      "reference": "Ex

2it [00:31, 14.34s/it]

...

⚠️ Skipping 3 John - No commentary found.

📜 Raw Gemini response for Jude:
```json
{
  "overview": "The book of Jude is a short, often overlooked, but important warning against false teachers who have infiltrated the church. Jude urges believers to contend for the true faith, which was delivered once for all to the saints, and to resist the corrupting influences of those who twist grace into license. He uses vivid imagery and historical examples to illustrate the judgment awaiting the ungodly. The letter also offers practical advice on how to care for those affected by false teaching, and how to remain steadfast in the love of God.",
  "key_scriptures": [
    "Jude 4: This verse identifies the false teachers as those who pervert God's grace into a license for immorality and deny Jesus Christ as Lord, highlighting the core issue of doctrinal corruption.",
    "Jude 5-7: These verses use examples from the Old Testament (Israel in the wilderness, fallen angels, Sodom and Gomorrah) to illu

65it [04:45,  2.73s/it]

✅ Updated: Jude

📜 Raw Gemini response for Revelation:
```json
{
  "overview": "The Book of Revelation is a unique and often debated book in the Bible, detailing a series of visions and prophecies concerning the end times.  It is a revelation of Jesus Christ, delivered through John, meant to encourage believers to endure hardship and remain faithful. The book depicts a cosmic struggle between good and evil, culminating in God's ultimate victory and the establishment of a new heaven and a new earth. It serves as both a warning against compromise and a source of hope for those facing persecution, promising reward for the faithful and judgment for the unrighteous.",
  "key_scriptures": [
    {
      "reference": "Revelation 1:3",
      "description": "Highlights the blessing attached to reading and heeding the words of Revelation, indicating the importance of understanding and applying its message."
    },
    {
      "reference": "Revelation 2:10",
      "description": "Exhorts believers

66it [05:04,  4.62s/it]

✅ Updated: Revelation


In [ ]:


# === Book Mapping for Combined Entries ===
BOOK_MAPPING = {
    "Judges": "Judges and Ruth",
    "Ruth": "Judges and Ruth",
    "1 Samuel": "1 and 2 Samuel",
    "2 Samuel": "1 and 2 Samuel",
    "1 Kings": "1 and 2 Kings",
    "2 Kings": "1 and 2 Kings",
    "1 Chronicles": "1 and 2 Chronicles",
    "2 Chronicles": "1 and 2 Chronicles",
    "Amos": "Amos and Hosea",
    "Hosea": "Amos and Hosea",
    "Jeremiah": "Jeremiah and Lamentations",
    "Lamentations": "Jeremiah and Lamentations",
    "Ezra": "Ezra and Nehemiah",
    "Nehemiah": "Ezra and Nehemiah",
    "1 Thessalonians": "1 and 2 Thessalonians",
    "2 Thessalonians": "1 and 2 Thessalonians",
    "1 Corinthians": "1 and 2 Corinthians",
    "2 Corinthians": "1 and 2 Corinthians",
    "1 Timothy": "1 and 2 Timothy and Titus",
    "2 Timothy": "1 and 2 Timothy and Titus",
    "Titus": "1 and 2 Timothy and Titus",
    "1 Peter": "1 and 2 Peter",
    "2 Peter": "1 and 2 Peter",
    "1 John": "1, 2 and 3 John",
    "2 John": "1, 2 and 3 John",
    "3 John": "1, 2 and 3 John"
}

PROMPT_TEMPLATE = """
You are a biblical scholar and summarizer rooted in strong commitment to biblical authority and Scripture’s central role. Based on the following commentary for the book of {book_name}, do the following:

1.  Write a 3–5 sentence **overview** of the book.
2.  Identify and list 2 to 3 **key scripture references** with brief descriptions.
3.  Write a 3–5 sentence **theological context** explaining the major doctrines and themes.

Commentary:
\"\"\"
{commentary_text}
\"\"\"

Return your response in a strict JSON format. The JSON object must contain the keys "overview", "key_scriptures", and "theological_context".
Do not include any text, explanations, or markdown formatting outside of the JSON object itself.
{{
  "overview": "...",
  "key_scriptures": [
    "...",
    "..."
  ],
  "theological_context": "..."
}}
"""

# === Revised Helper Function to Fix and Parse Gemini Output ===
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# === Main Process with Skip Logic ===
for book in tqdm(books_collection.find()):
    book_name = book["name"]

    # === ADJUSTMENT: CHECK IF FIELDS ALREADY EXIST ===
    # If all required fields are already in the document, skip processing.
    if ('overview' in book and 'key_scriptures' in book and 'theological_context' in book):
        print(f"⏩ Skipping {book_name}: Fields already exist.")
        continue # Moves to the next book in the loop

    # --- If fields don't exist, proceed with the original logic ---

    # === USE MAPPING TO GET COMMENTARY TITLE ===
    # Check if the book has a mapped combined title, otherwise use the book name
    commentary_title = BOOK_MAPPING.get(book_name, book_name)
    commentary_title_lower = commentary_title.lower()

    # Get all commentary chunks for this book using the mapped title (case-insensitive)
    chunks = commentary_collection.find({
        "$expr": { "$eq": [{ "$toLower": "$chapter_title" }, commentary_title_lower] }
    })

    all_text = " ".join(chunk["text"] for chunk in chunks)

    if not all_text.strip():
        print(f"⚠️ Skipping {book_name} - No commentary found for '{commentary_title}'.")
        continue

    # Format the prompt
    prompt = PROMPT_TEMPLATE.format(
        book_name=book_name,
        commentary_text=all_text[:200000] # Make sure your model's context window supports this
    )

    try:
        response = model.generate_content(prompt)
        raw_text = response.text.strip()

        print(f"\n📜 Raw Gemini response for {book_name} (using '{commentary_title}'):\n{raw_text}\n")

        generated_fields = parse_gemini_json(raw_text)

        # === Update MongoDB ===
        books_collection.update_one(
            {"_id": book["_id"]}, # Use the immutable _id for reliability
            {"$set": {
                "overview": generated_fields["overview"],
                "key_scriptures": generated_fields["key_scriptures"],
                "theological_context": generated_fields["theological_context"]
            }}
        )

        print(f"✅ Updated: {book_name} (using commentary from '{commentary_title}')")

    except Exception as e:
        print(f"❌ An error occurred processing {book_name}: {e}")
        continue

1it [00:00,  1.41it/s]

⏩ Skipping Genesis: Fields already exist.
⏩ Skipping Exodus: Fields already exist.
⏩ Skipping Leviticus: Fields already exist.
⏩ Skipping Numbers: Fields already exist.
⏩ Skipping Deuteronomy: Fields already exist.
⏩ Skipping Joshua: Fields already exist.

📜 Raw Gemini response for Judges (using 'Judges and Ruth'):
```json
{
  "overview": "The book of Judges depicts a dark period in Israel's history, characterized by a cyclical pattern of disobedience, oppression, repentance, and deliverance. It highlights Israel's failure to fully conquer the land and their subsequent compromise with Canaanite culture, leading to spiritual and moral decline. The absence of a righteous king results in societal chaos, with each person doing what seems right in their own eyes. Despite the flawed nature of the judges, God uses them to deliver Israel, demonstrating His mercy amidst their unfaithfulness.",
  "key_scriptures": [
    "Judges 2:11-19: This passage encapsulates the cyclical pattern of Israel's 

7it [00:18,  2.74s/it]

✅ Updated: Judges (using commentary from 'Judges and Ruth')

📜 Raw Gemini response for Ruth (using 'Judges and Ruth'):
```json
{
  "overview": "The book of Ruth, in contrast to Judges, tells a focused, linear story of God's providential care amidst hardship. Following Naomi's loss and return to Bethlehem with her Moabite daughter-in-law Ruth, the narrative unfolds a love story culminating in marriage between Ruth and Boaz. This union not only provides security and redemption for Naomi and Ruth but also establishes Ruth as an ancestor in the lineage of King David. The book serves as a testament to God's faithfulness and the importance of loyalty, obedience, and kindness.",
  "key_scriptures": [
    "Ruth 1:16-17: Ruth's declaration of loyalty to Naomi, her people, and her God demonstrates unwavering commitment and sets the stage for God's blessing.",
    "Ruth 4:13-17: The birth of Obed to Ruth and Boaz signifies God's provision and the continuation of the family line, ultimately leadin

8it [00:38,  5.74s/it]

...

⚠️ Skipping 1 John - No commentary found for '1, 2 and 3 John'.


63it [04:39,  3.21s/it]

⚠️ Skipping 2 John - No commentary found for '1, 2 and 3 John'.


66it [04:39,  4.24s/it]

⚠️ Skipping 3 John - No commentary found for '1, 2 and 3 John'.
⏩ Skipping Jude: Fields already exist.
⏩ Skipping Revelation: Fields already exist.


In [ ]:
# === Book Mapping for Combined Entries ===
BOOK_MAPPING = {

    "Song of Solomon": "Song of Songs",
    "1 John": "1, 2, And 3 John",
    "2 John": "1, 2, And 3 John",
    "3 John": "1, 2, And 3 John"
}

PROMPT_TEMPLATE = """
You are a biblical scholar and summarizer rooted in strong commitment to biblical authority and Scripture’s central role. Based on the following commentary for the book of {book_name}, do the following:

1.  Write a 3–5 sentence **overview** of the book.
2.  Identify and list 2 to 3 **key scripture references** with brief descriptions.
3.  Write a 3–5 sentence **theological context** explaining the major doctrines and themes.

Commentary:
\"\"\"
{commentary_text}
\"\"\"

Return your response in a strict JSON format. The JSON object must contain the keys "overview", "key_scriptures", and "theological_context".
Do not include any text, explanations, or markdown formatting outside of the JSON object itself.
{{
  "overview": "...",
  "key_scriptures": [
    "...",
    "..."
  ],
  "theological_context": "..."
}}
"""

# === Revised Helper Function to Fix and Parse Gemini Output ===
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# === Main Process with Skip Logic ===
for book in tqdm(books_collection.find()):
    book_name = book["name"]

    # === ADJUSTMENT: CHECK IF FIELDS ALREADY EXIST ===
    # If all required fields are already in the document, skip processing.
    if ('overview' in book and 'key_scriptures' in book and 'theological_context' in book):
        print(f"⏩ Skipping {book_name}: Fields already exist.")
        continue # Moves to the next book in the loop

    # --- If fields don't exist, proceed with the original logic ---

    # === USE MAPPING TO GET COMMENTARY TITLE ===
    # Check if the book has a mapped combined title, otherwise use the book name
    commentary_title = BOOK_MAPPING.get(book_name, book_name)
    commentary_title_lower = commentary_title.lower()

    # Get all commentary chunks for this book using the mapped title (case-insensitive)
    chunks = commentary_collection.find({
        "$expr": { "$eq": [{ "$toLower": "$chapter_title" }, commentary_title_lower] }
    })

    all_text = " ".join(chunk["text"] for chunk in chunks)

    if not all_text.strip():
        print(f"⚠️ Skipping {book_name} - No commentary found for '{commentary_title}'.")
        continue

    # Format the prompt
    prompt = PROMPT_TEMPLATE.format(
        book_name=book_name,
        commentary_text=all_text[:200000] # Make sure your model's context window supports this
    )

    try:
        response = model.generate_content(prompt)
        raw_text = response.text.strip()

        print(f"\n📜 Raw Gemini response for {book_name} (using '{commentary_title}'):\n{raw_text}\n")

        generated_fields = parse_gemini_json(raw_text)

        # === Update MongoDB ===
        books_collection.update_one(
            {"_id": book["_id"]}, # Use the immutable _id for reliability
            {"$set": {
                "overview": generated_fields["overview"],
                "key_scriptures": generated_fields["key_scriptures"],
                "theological_context": generated_fields["theological_context"]
            }}
        )

        print(f"✅ Updated: {book_name} (using commentary from '{commentary_title}')")

    except Exception as e:
        print(f"❌ An error occurred processing {book_name}: {e}")
        continue

1it [00:00,  1.06it/s]

⏩ Skipping Genesis: Fields already exist.
⏩ Skipping Exodus: Fields already exist.
⏩ Skipping Leviticus: Fields already exist.
⏩ Skipping Numbers: Fields already exist.
⏩ Skipping Deuteronomy: Fields already exist.
⏩ Skipping Joshua: Fields already exist.
⏩ Skipping Judges: Fields already exist.
⏩ Skipping Ruth: Fields already exist.
⏩ Skipping 1 Samuel: Fields already exist.
⏩ Skipping 2 Samuel: Fields already exist.
⏩ Skipping 1 Kings: Fields already exist.
⏩ Skipping 2 Kings: Fields already exist.
⏩ Skipping 1 Chronicles: Fields already exist.
⏩ Skipping 2 Chronicles: Fields already exist.
⏩ Skipping Ezra: Fields already exist.
⏩ Skipping Nehemiah: Fields already exist.
⏩ Skipping Esther: Fields already exist.
⏩ Skipping Job: Fields already exist.
⏩ Skipping Psalms: Fields already exist.
⏩ Skipping Proverbs: Fields already exist.
⏩ Skipping Ecclesiastes: Fields already exist.
⏩ Skipping Song of Solomon: Fields already exist.
⏩ Skipping Isaiah: Fields already exist.
⏩ Skipping Jeremi

62it [00:10,  5.95it/s]

✅ Updated: 1 John (using commentary from '1, 2, And 3 John')

📜 Raw Gemini response for 2 John (using '1, 2, And 3 John'):
```json
{
  "overview": "2 John is a brief, personal letter from the Apostle John to a specific woman (possibly named Kyria) and her children. John commends them for walking in truth and warns against showing hospitality to those who deny that Jesus Christ came in the flesh, a Gnostic heresy. He emphasizes the importance of both love and truth, urging discernment and adherence to the foundational teachings of Christ. The letter serves as a practical guide for maintaining sound doctrine and protecting the church from false teachers.",
  "key_scriptures": [
    {
      "reference": "2 John 1:7",
      "description": "Warns against those who deny Jesus Christ came in the flesh. This highlights the importance of the incarnation and refutes Docetism, a heresy prevalent at the time."
    },
    {
      "reference": "2 John 1:9",
      "description": "States that anyone w

63it [00:19,  2.77it/s]

✅ Updated: 2 John (using commentary from '1, 2, And 3 John')

📜 Raw Gemini response for 3 John (using '1, 2, And 3 John'):
```json
{
  "overview": "3 John is a short, personal letter from the Apostle John to Gaius, commending him for his hospitality towards traveling Christian teachers and missionaries. The letter also serves as a warning against the disruptive behavior of Diotrephes, who opposes John's authority and refuses to welcome fellow believers. John encourages Gaius to continue in his good works and promises to visit him soon. The central theme revolves around the importance of Christian fellowship and supporting those who spread the Gospel.",
  "key_scriptures": [
    {
      "reference": "3 John 1:4",
      "description": "John expresses his joy in hearing that his spiritual children are walking in truth, highlighting the importance of living according to God's Word."
    },
    {
      "reference": "3 John 1:11",
      "description": "John urges Gaius to imitate good rather

66it [00:26,  2.54it/s]

✅ Updated: 3 John (using commentary from '1, 2, And 3 John')
⏩ Skipping Jude: Fields already exist.
⏩ Skipping Revelation: Fields already exist.


### Enhancing Theological Concepts with Descriptions and Visual Themes

This section processes theological concepts from MongoDB by generating a concise one-sentence summary and selecting thematic colors using Gemini AI. Each concept is normalized into a kebab-case ID, then enriched with a description and two hex color values (primary and accent). The structured outputs are parsed and stored back in MongoDB, supporting better display and categorization of theological data.

In [ ]:

theology_collection = db["theology"]


# ========== NORMALIZE ID FUNCTION ==========
def kebab_case(text):
    text = text.lower()
    text = re.sub(r"[^\w\s-]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", "-", text.strip())  # Replace spaces with hyphens
    return text

# === Revised Helper Function to Fix and Parse Gemini Output ===
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# ========== DESCRIPTION PROMPT (UPDATED) ==========
DESCRIPTION_PROMPT = """
You are a theologian and summarizer rooted in Strong commitment to biblical authority and Scripture's central role. Given the following theological concept and its explanation, write a single-sentence description that captures the essence of "{concept}".

Concept: {concept}

Summary:
\"\"\"
{summary_text}
\"\"\"

Respond with just one sentence that describes what "{concept}" is.
"""

# ========== COLOR PROMPT ==========
COLOR_PROMPT = """
You are a creative designer. Based on the theological concept "{concept}" and this summary:

\"\"\"
{summary_text}
\"\"\"

Suggest two appropriate hex colors:
1. A main primary color for the concept (calm, bold, or thematic).
2. A related accent color (for emphasis or highlights).

Return your response in this JSON format:
{{
  "color": "#hexvalue",
  "arcColor": "#hexvalue"
}}
"""

# ========== MAIN LOOP ==========
for doc in tqdm(theology_collection.find()):
    concept = doc.get("concept", "").strip()
    summary_text = doc.get("summary", "").strip()

    if not concept or not summary_text:
        print(f"⚠️ Skipping document with missing concept or summary.")
        continue

    concept_id = kebab_case(concept)

    # ---------- 1. Generate One-Sentence Description ----------
    try:
        desc_prompt = DESCRIPTION_PROMPT.format(concept=concept, summary_text=summary_text)
        desc_response = model.generate_content(desc_prompt)
        desc_raw = desc_response.text.strip()
        print(f"\n📜 Description response for {concept}:\n{desc_raw}")

        # Clean and extract first sentence
        desc_cleaned = desc_raw.replace('"', '').replace("'", "").strip()
        description = desc_cleaned.split(".")[0].strip() + "."

    except Exception as e:
        print(f"❌ Error generating description for {concept}: {e}")
        continue

    # ---------- 2. Generate Colors ----------
    try:
        color_prompt = COLOR_PROMPT.format(concept=concept, summary_text=summary_text)
        color_response = model.generate_content(color_prompt)
        color_raw = color_response.text.strip()
        print(f"\n🎨 Color response for {concept}:\n{color_raw}")

        # ======== Parse Gemini JSON Safely =========
        color_data = parse_gemini_json(color_raw)
        color = color_data.get("color", "#3b82f6")
        arcColor = color_data.get("arcColor", "#fbbf24")

    except Exception as e:
        print(f"❌ Error generating colors for {concept}: {e}")
        color = "#3b82f6"
        arcColor = "#fbbf24"

    # ---------- 3. Update MongoDB ----------
    theology_collection.update_one(
        {"_id": doc["_id"]},
        {"$set": {
            "id": concept_id,
            "description": description,
            "color": color,
            "arcColor": arcColor
        }}
    )
    print(f"✅ Updated: {concept} → {concept_id}")

0it [00:00, ?it/s]


📜 Description response for The Holy Scriptures:
The Holy Scriptures are the sixty-six books of the Bible, God's inerrant, infallible, and verbally inspired written revelation to humanity, objectively and propositionally true, and the sole, authoritative rule for faith and practice, to be interpreted literally, grammatically, and historically.

🎨 Color response for The Holy Scriptures:
```json
{
  "color": "#2B3A67",
  "arcColor": "#F2D78D"
}
```


1it [00:05,  5.80s/it]

✅ Updated: The Holy Scriptures → the-holy-scriptures

📜 Description response for God:
God is the one infinite, all-knowing Spirit, perfect in all attributes, eternally existing as three co-equal Persons—Father, Son, and Holy Spirit—deserving all worship and obedience, as revealed in Scripture.

🎨 Color response for God:
```json
{
  "color": "#3B5998",
  "arcColor": "#FDB827"
}
```


2it [00:10,  4.93s/it]

...

❌ Error generating description for The Second Coming and the Millennial Reign: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


21it [00:58,  1.00it/s]

❌ Error generating description for The Judgment of the Lost: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


22it [00:59,  2.69s/it]

❌ Error generating description for Eternity: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


In [ ]:

theology_collection = db["theology"]


# ========== NORMALIZE ID FUNCTION ==========
def kebab_case(text):
    text = text.lower()
    text = re.sub(r"[^\w\s-]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", "-", text.strip())  # Replace spaces with hyphens
    return text

# === Revised Helper Function to Fix and Parse Gemini Output ===
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# ========== DESCRIPTION PROMPT (UPDATED) ==========
DESCRIPTION_PROMPT = """
You are a theologian and summarizer rooted in Strong commitment to biblical authority and Scripture's central role. Given the following theological concept and its explanation, write a single-sentence description that captures the essence of "{concept}".

Concept: {concept}

Summary:
\"\"\"
{summary_text}
\"\"\"

Respond with just one sentence that describes what "{concept}" is.
"""

# ========== COLOR PROMPT ==========
COLOR_PROMPT = """
You are a creative designer. Based on the theological concept "{concept}" and this summary:

\"\"\"
{summary_text}
\"\"\"

Suggest two appropriate hex colors:
1. A main primary color for the concept (calm, bold, or thematic).
2. A related accent color (for emphasis or highlights).

Return your response in this JSON format:
{{
  "color": "#hexvalue",
  "arcColor": "#hexvalue"
}}
"""

# ========== MAIN LOOP ==========
for doc in tqdm(theology_collection.find()):
    concept = doc.get("concept", "").strip()
    summary_text = doc.get("summary", "").strip()

    if not concept or not summary_text:
        print(f"⚠️ Skipping document with missing concept or summary.")
        continue

    concept_id = kebab_case(concept)

    # ---------- Check if document already has all required fields ----------
    if (doc.get("id") and doc.get("description") and
        doc.get("color") and doc.get("arcColor")):
        print(f"⏭️ Skipping {concept} - already has all fields")
        continue

    # ---------- 1. Generate One-Sentence Description ----------
    try:
        desc_prompt = DESCRIPTION_PROMPT.format(concept=concept, summary_text=summary_text)
        desc_response = model.generate_content(desc_prompt)
        desc_raw = desc_response.text.strip()
        print(f"\n📜 Description response for {concept}:\n{desc_raw}")

        # Clean and extract first sentence
        desc_cleaned = desc_raw.replace('"', '').replace("'", "").strip()
        description = desc_cleaned.split(".")[0].strip() + "."

    except Exception as e:
        print(f"❌ Error generating description for {concept}: {e}")
        continue

    # ---------- 2. Generate Colors ----------
    try:
        color_prompt = COLOR_PROMPT.format(concept=concept, summary_text=summary_text)
        color_response = model.generate_content(color_prompt)
        color_raw = color_response.text.strip()
        print(f"\n🎨 Color response for {concept}:\n{color_raw}")

        # ======== Parse Gemini JSON Safely =========
        color_data = parse_gemini_json(color_raw)
        color = color_data.get("color", "#3b82f6")
        arcColor = color_data.get("arcColor", "#fbbf24")

    except Exception as e:
        print(f"❌ Error generating colors for {concept}: {e}")
        color = "#3b82f6"
        arcColor = "#fbbf24"

    # ---------- 3. Update MongoDB ----------
    theology_collection.update_one(
        {"_id": doc["_id"]},
        {"$set": {
            "id": concept_id,
            "description": description,
            "color": color,
            "arcColor": arcColor
        }}
    )
    print(f"✅ Updated: {concept} → {concept_id}")

1it [00:00,  1.42it/s]

⏭️ Skipping The Holy Scriptures - already has all fields
⏭️ Skipping God - already has all fields
⏭️ Skipping God The Father - already has all fields
⏭️ Skipping God The Son - already has all fields
⏭️ Skipping God the Holy Spirit - already has all fields
⏭️ Skipping Man - already has all fields
⏭️ Skipping Salvation - already has all fields
⏭️ Skipping Election - already has all fields
⏭️ Skipping Regeneration - already has all fields
⏭️ Skipping Justification - already has all fields
⏭️ Skipping Sanctification - already has all fields
⏭️ Skipping Security - already has all fields
⏭️ Skipping Separation - already has all fields
⏭️ Skipping The Church - already has all fields
⏭️ Skipping Holy Angels - already has all fields
⏭️ Skipping Fallen Angels - already has all fields
⏭️ Skipping Death - already has all fields
⏭️ Skipping The Rapture of the Church - already has all fields
⏭️ Skipping The Tribulation Period - already has all fields
⏭️ Skipping The Second Coming and the Millennial 

22it [00:07,  3.06it/s]

✅ Updated: Eternity → eternity


### Standardizing Book Names in Reference Statistics

This section ensures consistency in Bible book references by mapping abbreviations (e.g., *Gen.*) to their full canonical names (e.g., *Genesis*). It scans all theology documents, applies the `book_alias_map` to normalize keys in the `book_reference_stats` field, merges duplicates, and updates the database only if changes are detected.

In [ ]:


collection = db["theology"]

# Abbreviation to canonical name mapping
book_alias_map = {
    'Gen.': 'Genesis',
    'Ex.': 'Exodus',
    'Lev.': 'Leviticus',
    'Num.': 'Numbers',
    'Deut.': 'Deuteronomy',
    'Josh.': 'Joshua',
    'Judg.': 'Judges',
    '1 Sam.': '1 Samuel',
    '2 Sam.': '2 Samuel',
    '1 Kings': '1 Kings',
    '2 Kings': '2 Kings',
    '1 Chron.': '1 Chronicles',
    '2 Chron.': '2 Chronicles',
    'Ezra': 'Ezra',
    'Neh.': 'Nehemiah',
    'Est.': 'Esther',
    'Job': 'Job',
    'Ps.': 'Psalms',
    'Psalm': 'Psalms',
    'Psalms': 'Psalms',
    'Prov.': 'Proverbs',
    'Eccl.': 'Ecclesiastes',
    'Song of Sol.': 'Song of Solomon',
    'Isa.': 'Isaiah',
    'Jer.': 'Jeremiah',
    'Lam.': 'Lamentations',
    'Ezek.': 'Ezekiel',
    'Dan.': 'Daniel',
    'Hos.': 'Hosea',
    'Joel': 'Joel',
    'Amos': 'Amos',
    'Obad.': 'Obadiah',
    'Jonah': 'Jonah',
    'Mic.': 'Micah',
    'Nah.': 'Nahum',
    'Hab.': 'Habakkuk',
    'Zech.': 'Zechariah',
    'Zeph.': 'Zephaniah',
    'Hag.': 'Haggai',
    'Mal.': 'Malachi',
    'Matt.': 'Matthew',
    'Mark': 'Mark',
    'Luke': 'Luke',
    'John': 'John',
    'Acts': 'Acts',
    'Rom.': 'Romans',
    '1 Cor.': '1 Corinthians',
    '2 Cor.': '2 Corinthians',
    'Gal.': 'Galatians',
    'Eph.': 'Ephesians',
    'Phil.': 'Philippians',
    'Col.': 'Colossians',
    '1 Thess.': '1 Thessalonians',
    '2 Thess.': '2 Thessalonians',
    '1 Tim.': '1 Timothy',
    '2 Tim.': '2 Timothy',
    'Titus': 'Titus',
    'Philem.': 'Philemon',
    'Heb.': 'Hebrews',
    'James': 'James',
    '1 Pet.': '1 Peter',
    '2 Pet.': '2 Peter',
    '1 John': '1 John',
    '2 John': '2 John',
    '3 John': '3 John',
    'Jude': 'Jude',
    'Rev.': 'Revelation'
}

# Go through each document in the collection
for doc in collection.find({}, {"book_reference_stats": 1}):
    original = doc.get("book_reference_stats", {})
    updated = {}

    changed = False
    for key, value in original.items():
        # Map to canonical name
        new_key = book_alias_map.get(key, key)
        if new_key in updated:
            updated[new_key] += value  # Combine stats if duplicate canonical name
        else:
            updated[new_key] = value
        if new_key != key:
            changed = True

    # If changes occurred, update the document
    if changed:
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"book_reference_stats": updated}}
        )
        print(f"Updated document {doc['_id']} with standardized book names.")
print("✅ All applicable documents have been updated.")

Updated document 684c15ecdc45f4c1ffcbf3a7 with standardized book names.
Updated document 684c1613dc45f4c1ffcbf3a8 with standardized book names.
Updated document 684c1662dc45f4c1ffcbf3a9 with standardized book names.
Updated document 684c16f5dc45f4c1ffcbf3ac with standardized book names.
Updated document 684c1755dc45f4c1ffcbf3ae with standardized book names.
Updated document 684c1798dc45f4c1ffcbf3b0 with standardized book names.
Updated document 684c1d15dc45f4c1ffcbf3b9 with standardized book names.
✅ All applicable documents have been updated.


In [ ]:

collection = db["theology"]

# Initialize a set to collect unique keys
unique_books = set()

# Iterate through all documents
for doc in collection.find({}, {"book_reference_stats": 1}):
    if "book_reference_stats" in doc:
        unique_books.update(doc["book_reference_stats"].keys())

# Convert to sorted list and print
unique_books = sorted(unique_books)
print(unique_books)

['1 Chronicles', '1 Corinthians', '1 John', '1 Peter', '1 Thessalonians', '1 Timothy', '2 Corinthians', '2 John', '2 Peter', '2 Thessalonians', '2 Timothy', 'Acts', 'Colossians', 'Daniel', 'Deuteronomy', 'Ephesians', 'Exodus', 'Ezekiel', 'Galatians', 'Genesis', 'Habakkuk', 'Hebrews', 'Isaiah', 'James', 'Jeremiah', 'John', 'Jude', 'Luke', 'Matthew', 'Micah', 'Philippians', 'Psalms', 'Revelation', 'Romans', 'Titus', 'Zechariah']


### Linking Theological Concepts to Biblical Books and Events

This section enriches each theological concept by identifying related biblical books and assigning prominence based on reference frequency. It then uses Gemini to generate key biblical events linked to each concept. The script matches book names from a separate collection, maps prominence scores (1–5), fetches 2–4 major events, and stores the structured `connections` data—including book ID, prominence, and events—back into MongoDB.

In [ ]:

theology_collection = db["theology"]
books_collection = db["books"]


# ========== PROMPT FOR EVENTS ==========
EVENT_PROMPT = """
You are a
You are a biblical scholar rooted in strong commitment to biblical authority and Scripture’s central role. Based on the following theological concept and its summary, identify 2–4 biblical events or milestones that are strongly associated with this concept. These events should correspond to what the Bible says or what the concept implies.

Concept: "{concept}"

Summary:
\"\"\"
{summary}
\"\"\"

Return your answer in this JSON format:
{{
  "events": [
    "Event 1",
    "Event 2",
    "..."
  ]
}}
"""

# ========== ENHANCED JSON PARSER ==========
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# ========== PERCENTAGE → PROMINENCE CONVERSION ==========
def percent_to_prominence(percent):
    return max(1, min(5, round(percent / 20)))  # ensures values between 1 and 5

# ========== MAIN PROCESS ==========
for doc in tqdm(theology_collection.find()):
    concept = doc.get("concept", "").strip()
    summary = doc.get("summary", "").strip()
    ref_stats = doc.get("book_reference_stats", {})

    if not concept or not summary or not ref_stats:
        print(f"⚠️ Skipping {concept or 'unnamed'} due to missing data.")
        continue

    # === 1. Build Book Connections ===
    connections = []
    for book_name, percent in ref_stats.items():
        book = books_collection.find_one({ "name": {"$regex": f"^{re.escape(book_name)}$", "$options": "i"} })
        if not book:
            print(f"⚠️ Book not found: {book_name}")
            continue

        book_id = book["id"]
        prominence = percent_to_prominence(percent)

        # Placeholder events to be filled later
        connections.append({
            "bookId": book_id,
            "prominence": prominence,
            "events": []
        })

    if not connections:
        print(f"⚠️ No valid books found for {concept}")
        continue

    # === 2. Generate Events from Summary ===
    try:
        prompt = EVENT_PROMPT.format(concept=concept, summary=summary)
        response = model.generate_content(prompt)
        print(f"\n📜 Gemini event response for {concept}:\n{response.text.strip()}")

        parsed = parse_gemini_json(response.text)
        events = parsed.get("events", [])

        for c in connections:
            c["events"] = events

    except Exception as e:
        print(f"❌ Error generating events for {concept}: {e}")
        continue

    # === 3. Update Theology Document ===
    theology_collection.update_one(
        {"_id": doc["_id"]},
        {"$set": {"connections": connections}}
    )

    print(f"✅ Updated connections for: {concept}")

0it [00:00, ?it/s]


📜 Gemini event response for The Holy Scriptures:
```json
{
  "events": [
    "The Giving of the Law at Mount Sinai: This event underscores the concept of God's written revelation to humanity. God directly gave Moses the Ten Commandments inscribed on tablets of stone (Exodus 20; 31:18; 34:1), establishing a foundational written word that served as a covenant between God and His people. It demonstrates God's direct and authoritative communication inscripturated for preservation and instruction.",
    "The Prophetic Ministry: The prophets throughout the Old Testament, like Isaiah, Jeremiah, and Ezekiel, received divine messages through the Holy Spirit (2 Peter 1:20-21) and faithfully recorded them. Their writings are considered God's word, verbally inspired (2 Timothy 3:16) and intended to guide and correct God's people. Their messages highlight God's ongoing communication and intervention in human history through inspired, written words.",
    "Jesus' Affirmation of Scripture: Jesus con

1it [00:09,  9.79s/it]

✅ Updated connections for: The Holy Scriptures

📜 Gemini event response for God:
```json
{
  "events": [
    "The Creation Account (Genesis 1-2): Demonstrates God's power, wisdom, and role as the sole creator of all things, reflecting His infinite nature and existence before all else.",
    "The Giving of the Law at Mount Sinai (Exodus 19-20): Highlights God's holiness, justice, and authority as the one true God who establishes a covenant with His people and gives them His commandments.",
    "The Incarnation of Jesus Christ (Gospels): Underscores the Trinitarian nature of God, with the eternal Son taking on human flesh, revealing God's love, grace, and plan of salvation for humanity.",
     "The Day of Pentecost (Acts 2): Showcases the outpouring of the Holy Spirit, demonstrating the active presence and power of God in the lives of believers and the continuation of His work in the world through the third person of the Trinity."
  ]
}
```


2it [00:13,  6.47s/it]

...

✅ Updated connections for: The Second Coming and the Millennial Reign

📜 Gemini event response for The Judgment of the Lost:
```json
{
  "events": [
    "Satan's Release and Final Rebellion (Revelation 20:7-9): This event directly precedes the final judgment of the lost, as Satan's deception gathers the nations for a final battle, setting the stage for divine intervention and ultimate judgment.",
    "The Great White Throne Judgment (Revelation 20:11-15): This is the central event depicting the judgment of the unsaved dead. The Bible describes the resurrection of all who are not found written in the Book of Life and their subsequent judgment based on their works, leading to their consignment to the lake of fire.",
    "The Casting of Satan into the Lake of Fire (Revelation 20:10): While not directly the judgment of humanity, Satan's eternal punishment in the lake of fire is inextricably linked to the judgment of the lost, highlighting the ultimate defeat of evil and the eternal consequ

21it [02:06,  4.76s/it]

✅ Updated connections for: The Judgment of the Lost

📜 Gemini event response for Eternity:
```json
{
  "events": [
    "The Dissolution of the Old Earth and Creation of the New Earth: This event, described in 2 Peter 3:10 and Revelation 21:1, marks a definitive break with the current, flawed creation and the establishment of a perfect, righteous dwelling place for the redeemed in eternity.",
    "The Descent of the New Jerusalem: Revelation 21:2 depicts the New Jerusalem descending from heaven to earth, signifying the ultimate union of God and His people. This event represents the culmination of God's redemptive plan and the establishment of eternal fellowship.",
    "Christ Delivering the Kingdom to God the Father: As outlined in 1 Corinthians 15:24-28, Christ's act of handing over the kingdom to God the Father demonstrates the ultimate purpose of redemption – the eternal reign of the Triune God. This milestone signifies the completion of Christ's mediatorial role and the full manifes

22it [02:11,  5.97s/it]

✅ Updated connections for: Eternity


In [ ]:

theology_collection = db["theology"]


# Google AI Configuration
EMBEDDING_MODEL = "models/text-embedding-004"  # The model for generating embeddings

def generate_embedding(text):
    """Generates an embedding for the given text using Google AI."""
    try:
        # The 'task_type' is crucial for retrieval/semantic search tasks
        result = genai.embed_content(
            model=EMBEDDING_MODEL,
            content=text,
            task_type="RETRIEVAL_DOCUMENT"
        )
        return result['embedding']
    except Exception as e:
        print(f"An error occurred during embedding generation: {e}")
        return None

# ========== PROMPT FOR STANDARDIZING REFERENCES ==========
STANDARDIZE_PROMPT = """
You are a biblical scholar and editor rooted in strong commitment to biblical authority and scriptures central role. Your task is to standardize biblical references in theological summaries by converting specific verse citations to general book references.

Convert detailed verse references like "(Gen 2:7, 15–25; James 3:9)" to simplified book references like "as seen in Genesis and James."

Original Summary:
\"\"\"
{summary}
\"\"\"

Please return the standardized summary with verse references converted to book references. Keep all other content exactly the same.

Return your response in this JSON format:
{{
  "standardized_summary": "..."
}}
"""

# ========== ENHANCED JSON PARSER ==========
def parse_gemini_json(raw_text: str):
    """
    Extracts, cleans, and parses a JSON object from a raw string from a model.
    Handles responses wrapped in markdown backticks and other common LLM artifacts.
    """
    # 1. Extract JSON block from markdown ```json ... ``` or ``` ... ```
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", raw_text, re.DOTALL)

    if match:
        cleaned_text = match.group(1).strip()
    else:
        # 2. If no backticks, find the first '{' to the last '}'
        # This handles cases where the model returns a raw JSON object without markdown.
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        if start_index == -1 or end_index == -1:
            raise ValueError("🛑 No valid JSON object found in the response.")
        cleaned_text = raw_text[start_index : end_index + 1]

    # 3. Clean up common formatting issues
    # Replace slanted or "smart" quotes with standard double quotes
    cleaned_text = cleaned_text.replace(""", '"').replace(""", '"')
    # This replacement is left out as it can corrupt apostrophes (e.g., God's)
    # cleaned_text = cleaned_text.replace("'", "'").replace("'", "'")

    # Remove trailing commas from lists (e.g., [1, 2, ]) and objects (e.g., {a:1,})
    cleaned_text = re.sub(r",\s*\]", "]", cleaned_text)
    cleaned_text = re.sub(r",\s*}", "}", cleaned_text)

    # 4. Attempt to parse the cleaned JSON
    try:
        return json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        # If parsing fails, raise a detailed error for easier debugging
        error_message = f"🛑 JSON parsing failed after cleanup: {e}"
        # For debugging, you can uncomment the next line to see the failed text
        # print(f"--- Failed Text ---\n{cleaned_text}\n---------------------")
        raise ValueError(error_message) from e

# ========== MAIN PROCESS ==========
for doc in tqdm(theology_collection.find()):
    concept = doc.get("concept", "").strip()
    summary = doc.get("summary", "").strip()

    # Skip if no concept or summary
    if not concept or not summary:
        print(f"⚠️ Skipping {concept or 'unnamed'} due to missing concept or summary.")
        continue

    # Skip if embedding already exists
    if "embedding" in doc:
        print(f"⏩ Skipping {concept}: Embedding already exists.")
        continue

    try:
        # === 1. Standardize Biblical References ===
        print(f"\n🔄 Processing {concept}...")

        prompt = STANDARDIZE_PROMPT.format(summary=summary)
        response = model.generate_content(prompt)

        print(f"📜 Standardization response for {concept}:\n{response.text.strip()}\n")

        parsed = parse_gemini_json(response.text)
        standardized_summary = parsed.get("standardized_summary", summary)

        # === 2. Generate Embedding ===
        print(f"🧠 Generating embedding for {concept}...")
        embedding = generate_embedding(standardized_summary)

        if embedding is None:
            print(f"❌ Failed to generate embedding for {concept}")
            continue

        # === 3. Update Document ===
        theology_collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {
                "standardized_summary": standardized_summary,
                "embedding": embedding
            }}
        )

        print(f"✅ Updated {concept} with standardized summary and embedding")

    except Exception as e:
        print(f"❌ Error processing {concept}: {e}")
        continue

print("\n🎉 Finished processing all theology documents!")

0it [00:00, ?it/s]


🔄 Processing The Holy Scriptures...
📜 Standardization response for The Holy Scriptures:
```json
{
  "standardized_summary": "We teach that the Bible is God’s written revelation to man, and thus the 66 books of the Bible given to us by the Holy Spirit constitute the plenary (inspired equally in all parts) word of God (as seen in 1 Corinthians and 2 Peter). We teach that the word of God is an objective, propositional revelation (as seen in 1 Corinthians and 1 Thessalonians), verbally inspired in every word (as seen in 2 Timothy), absolutely inerrant in the original documents, infallible, and God-breathed. We teach the literal, grammatical-historical interpretation of Scripture, which affirms the belief that the opening chapters of Genesis present creation in six literal days (as seen in Genesis and Exodus). We teach that the Bible constitutes the only infallible rule of faith and practice (as seen in Matthew, John, 1 Corinthians, 2 Timothy, Hebrews, and 2 Peter). We teach that God spoke

1it [00:07,  7.64s/it]

✅ Updated The Holy Scriptures with standardized summary and embedding

🔄 Processing God...
📜 Standardization response for God:
```json
{
  "standardized_summary": "We teach that there is but one living and true God (as seen in Deuteronomy, Isaiah, and 1 Corinthians), an infinite, all-knowing Spirit (John), perfect in all his attributes, one in essence, eternally existing in three Persons—Father, Son, and Holy Spirit (Matthew and 2 Corinthians)—each equally deserving worship and obedience."
}
```

🧠 Generating embedding for God...


2it [00:10,  4.65s/it]

...

✅ Updated The Second Coming and the Millennial Reign with standardized summary and embedding

🔄 Processing The Judgment of the Lost...
📜 Standardization response for The Judgment of the Lost:
```json
{
  "standardized_summary": "We teach that following the release of Satan after the thousand-year reign of Christ (as seen in Revelation), Satan will deceive the nations of the earth and gather them to battle against the saints and the beloved city, at which time Satan and his army will be devoured by fire from heaven (as seen in Revelation). Following this, Satan will be thrown into the lake of fire and brimstone (as seen in Matthew and Revelation) whereupon Christ, who is the judge of all men (as seen in John), will resurrect and judge the great and small at the great white throne judgment.We teach that this resurrection of the unsaved dead to judgment will be a physical resurrection, whereupon receiving their judgment (as seen in John), they will be committed to an eternal conscious pun

21it [01:20,  3.25s/it]

✅ Updated The Judgment of the Lost with standardized summary and embedding

🔄 Processing Eternity...
📜 Standardization response for Eternity:
```json
{
  "standardized_summary": "We teach that after the closing of the Millennium, the temporary release of  Satan, and the judgment of unbelievers (as seen in 2 Thessalonians and Revelation), the saved  will enter the eternal state of glory with God, after which the elements of this  earth are to be dissolved (as seen in 2 Peter) and replaced with a new earth wherein  only righteousness dwells (as seen in Ephesians and Revelation). Following this, the  heavenly city will come down out of heaven (as seen in Revelation) and will be the dwelling  place of the saints, where they will enjoy forever fellowship with God and one  another (as seen in John and Revelation). Our Lord Jesus Christ, having fulfilled his  redemptive mission, will then deliver up the kingdom to God the Father (as seen in 1 Corinthians) that in all spheres the triune God ma

22it [01:24,  3.83s/it]

✅ Updated Eternity with standardized summary and embedding

🎉 Finished processing all theology documents!
